In [94]:
import  os

In [95]:
%pwd

'/Users/sahilsingh/Desktop/ML-TextSummarizer/Text-Summarizer-'

In [83]:
os.chdir('ML-TextSummarizer/Text-Summarizer-')

In [96]:
%pwd

'/Users/sahilsingh/Desktop/ML-TextSummarizer/Text-Summarizer-'

In [98]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    source_URL : str
    local_data_file : Path
    unzip_dir : Path


In [99]:
from Textsummarizer.constants import *
from Textsummarizer.utils.common import read_yaml, create_directories

In [101]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [102]:
import os
import urllib.request as request
import zipfile
from Textsummarizer.logging import logger
from Textsummarizer.utils.common import get_size

In [107]:
import os
import ssl
import certifi
import urllib.request

class DataIngestion:
    def __init__(self, config):
        self.config = config

    def download_file(self):
        url = self.config.source_URL
        local_path = self.config.local_data_file

        # Create SSL context using certifi
        ssl_context = ssl.create_default_context(cafile=certifi.where())

        # Download the file securely
        with urllib.request.urlopen(url, context=ssl_context) as response:
            with open(local_path, 'wb') as out_file:
                out_file.write(response.read())

        print(f"✅ File downloaded successfully to {local_path}")

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [108]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-25 17:56:47,856 : INFO : common : yaml file config/config.yaml loaded successfully]
[2025-08-25 17:56:47,859 : INFO : common : yaml file params.yaml loaded successfully]
[2025-08-25 17:56:47,859 : INFO : common : created directory at : artifacts]
[2025-08-25 17:56:47,861 : INFO : common : created directory at : artifacts/data_ingestion]
✅ File downloaded successfully to artifacts/data_ingestion/data.zip
